In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-tachygraphy/Tachygraphy_dataset_main.csv


**Text Preprocessing**

In [2]:
# To ignore warinings
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv("/kaggle/input/dataset-tachygraphy/Tachygraphy_dataset_main.csv")
df.shape

(4958, 3)

In [4]:
df.head()

,Text,Meaning,Sentiment
0,Last session of the day http://twitpic.com/67ezh,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely --...,Shanghai is also really exciting (precisely --...,positive
2,submit the report ASAP!,submit the report as soon as possilbe,negative
3,happy bday!,Happy Birthday!,positive
4,The OGs - I like it!!,The original gangsters - i like it!,positive


In [5]:
df=df.drop('Meaning',axis=1)
df = df.rename(columns={'Text': 'tweet'})
df = df.rename(columns={'Sentiment': 'class'})
df['class'] = df['class'].str.lower()
sentiment_counts = df['class'].value_counts()
sentiment_counts

class
neutral     1743
positive    1643
negative    1572
Name: count, dtype: int64

In [6]:
df.isnull().sum()

tweet    1
class    0
dtype: int64

In [7]:
df=df.fillna('')

In [8]:
import re
from bs4 import BeautifulSoup

def text_cleaning(text):
    text = str(text)
    soup = BeautifulSoup(text, "html.parser")    ###removing html tages
    text = re.sub(r'\[[^]]*\]', '', soup.get_text())      ##removing text within square brackets
    pattern = r"[^a-zA-Z0-9\s,']"                  # Removing unwanted characters
    text = re.sub(pattern, '', text)
    url_pattern = r'http\S+|www\S+'            ###removing urls
    text = re.sub(url_pattern, '', text)
    return text

In [9]:
df['tweet']=df['tweet'].apply(text_cleaning).tolist()

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import re

2024-08-24 14:46:00.171332: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-24 14:46:00.171439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-24 14:46:00.292559: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [13]:
df['tweet'].dtype

dtype('O')

In [14]:
df['class'].dtype

dtype('O')

In [15]:
df.tail(10)

,tweet,class
4948,time flies by when you're facetiming the bae,positive
4949,U dont know how I feel about this,negative
4950,come to my job I'll give you a baconator lol,positive
4951,Rihanna just made Maya Angelou her avi you're...,negative
4952,im the lil bitch in the theaters tryna hold ba...,negative
4953,make a pet face wtf wrong with me tonight haha,negative
4954,I dnt care anymore Boyz ain't worth d drama,negative
4955,No relationship is perfect tho me bae goo from...,negative
4956,Over here tryna get my nail polishes and shit lol,negative
4957,No one was loved d way i luv U,positive


In [16]:
df = df[df['class'] != "neutral"]  ###droping neutral columns
df['tweet'] = df['tweet'].apply(lambda x: x.lower())

print(df[ df['class'] == 'positive'].size)
print(df[ df['class'] == 'negative'].size)

for idx,row in df.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['tweet'].values)
X = tokenizer.texts_to_sequences(df['tweet'].values)
X = pad_sequences(X)

3286
3144


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import RMSprop
from keras_tuner import RandomSearch

In [18]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2024-08-24 14:46:11--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-08-24 14:46:11--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-08-24 14:46:11--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: 'glove.6B.zip'

glov

In [19]:
embedding_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs


In [20]:
# max_features = 5000  
# embed_dim = 128 
# lstm_out = 64  
# batch_size = 32
# epochs = 20
# learning_rate = 0.001

In [21]:
max_features = 5000  
embed_dim = 128 
lstm_out = 64  
batch_size = 64
epochs = 20
initial_learning_rate = 0.0005

In [22]:
embedding_dim = 100
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_features:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [23]:
# # from tensorflow.keras.models import Sequential
# # from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense


# # Model
# model = Sequential()
# model.add(Embedding(max_features, embedding_dim, embeddings_initializer=Constant(embedding_matrix),
#                     input_length=X.shape[1], trainable=True))
# model.add(SpatialDropout1D(0.3))
# model.add(Bidirectional(LSTM(128, return_sequences=True)))
# model.add(LSTM(64, dropout=0.4, recurrent_dropout=0.4))
# model.add(Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
# model.add(Dropout(0.4))
# model.add(Dense(2, activation='softmax'))  # Assuming 2 classes for classification

# # model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# # print(model.summary())

In [24]:
tf.random.set_seed(42)

In [25]:
# optimizer = Adam(learning_rate=initial_learning_rate)

In [26]:
# model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])

In [27]:
# print(model.summary())

In [28]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=0.00001, verbose=1)
# model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')


In [29]:
Y = pd.get_dummies(df['class']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2572, 29) (2572, 2)
(643, 29) (643, 2)


In [30]:
validation_size = 500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

# MODEL

In [31]:
# Define the hyperparameter tuning function
def build_model(hp):
    model = Sequential()
    
    # Embedding layer with pre-trained GloVe embeddings
    model.add(Embedding(max_features,
                        embedding_dim,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=X.shape[1],
                        trainable=True))
    
    # Tuning SpatialDropout1D rate
    model.add(SpatialDropout1D(hp.Float('spatial_dropout', min_value=0.1, max_value=0.5, step=0.1)))
    
    # Bidirectional LSTM layer with tunable units
    model.add(Bidirectional(LSTM(units=hp.Int('lstm_units_1', min_value=64, max_value=128, step=32), 
                                 return_sequences=True)))
    
    # Second LSTM layer with dropout and recurrent dropout
    model.add(LSTM(units=hp.Int('lstm_units_2', min_value=32, max_value=64, step=32),
                   dropout=hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1),
                   recurrent_dropout=hp.Float('recurrent_dropout', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Dense layer with L2 regularization
    model.add(Dense(units=hp.Int('dense_units', min_value=32, max_value=64, step=16),
                    activation='relu',
                    kernel_regularizer=tf.keras.regularizers.l2(0.01)))
    
    # Dropout layer
    model.add(Dropout(hp.Float('dropout_dense', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Output layer
    model.add(Dense(2, activation='softmax'))
    
    # Compile the model
#     model.compile(
#         optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
#         loss='categorical_crossentropy',
#         metrics=['accuracy']
#     )
    
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [32]:
# Instantiate the RandomSearch tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,  # Number of different models to try
    executions_per_trial=4,  # Number of times to train each model
    directory='my_dir',
    project_name='lstm_hyperparameter_tuning'
)

In [33]:

# Display search space summary
tuner.search_space_summary()

Search space summary
Default search space size: 8
spatial_dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
lstm_units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
lstm_units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 32, 'sampling': 'linear'}
dropout (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
recurrent_dropout (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
dropout_dense (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_valu

In [34]:
# batch_size = 32
# model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)

# Training
# history = model.fit(X_train, Y_train,
#                     epochs=epochs,
#                     batch_size=batch_size,
#                     validation_data=(X_test, Y_test),
#                     verbose=2,
#                     callbacks=[early_stopping, reduce_lr, model_checkpoint])

In [35]:
# Perform hyperparameter tuning
tuner.search(X_train, Y_train, 
             epochs=20, 
             validation_data=(X_test, Y_test), 
             batch_size=64, 
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])


Trial 20 Complete [00h 02m 55s]
val_accuracy: 0.779720276594162

Best val_accuracy So Far: 0.8111888021230698
Total elapsed time: 00h 39m 26s


## BEST HYPERPARAMS AS PERR TUNER

In [36]:
# Display best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=3)[0]
print(f"""
The hyperparameter search is complete. 
Optimal SpatialDropout1D rate: {best_hps.get('spatial_dropout')}
Optimal LSTM units for first layer: {best_hps.get('lstm_units_1')}
Optimal LSTM units for second layer: {best_hps.get('lstm_units_2')}
Optimal dropout rate for second LSTM layer: {best_hps.get('dropout')}
Optimal recurrent dropout rate for second LSTM layer: {best_hps.get('recurrent_dropout')}
Optimal dense layer units: {best_hps.get('dense_units')}
Optimal dropout rate for dense layer: {best_hps.get('dropout_dense')}
Optimal learning rate: {best_hps.get('learning_rate')}
""")



The hyperparameter search is complete. 
Optimal SpatialDropout1D rate: 0.5
Optimal LSTM units for first layer: 128
Optimal LSTM units for second layer: 64
Optimal dropout rate for second LSTM layer: 0.4
Optimal recurrent dropout rate for second LSTM layer: 0.30000000000000004
Optimal dense layer units: 48
Optimal dropout rate for dense layer: 0.4
Optimal learning rate: 0.0014505782975686376



## SECOND BEST HYPERPARAMS AS PER TUNER

In [37]:
# Display 2nd best hyperparameters
best_hps2 = tuner.get_best_hyperparameters(num_trials=3)[1]
print(f"""
The hyperparameter search is complete. 
Optimal SpatialDropout1D rate: {best_hps2.get('spatial_dropout')}
Optimal LSTM units for first layer: {best_hps2.get('lstm_units_1')}
Optimal LSTM units for second layer: {best_hps2.get('lstm_units_2')}
Optimal dropout rate for second LSTM layer: {best_hps2.get('dropout')}
Optimal recurrent dropout rate for second LSTM layer: {best_hps2.get('recurrent_dropout')}
Optimal dense layer units: {best_hps2.get('dense_units')}
Optimal dropout rate for dense layer: {best_hps2.get('dropout_dense')}
Optimal learning rate: {best_hps2.get('learning_rate')}
""")


The hyperparameter search is complete. 
Optimal SpatialDropout1D rate: 0.4
Optimal LSTM units for first layer: 64
Optimal LSTM units for second layer: 32
Optimal dropout rate for second LSTM layer: 0.2
Optimal recurrent dropout rate for second LSTM layer: 0.2
Optimal dense layer units: 48
Optimal dropout rate for dense layer: 0.4
Optimal learning rate: 0.0009181671251157912



In [38]:
# # Display best hyperparameters
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
# print(f"""
# The hyperparameter search is complete. 
# Optimal dropout rate: {best_hps.get('dropout_rate')}
# Optimal LSTM units: {best_hps.get('lstm_units')}
# Optimal dense layer units: {best_hps.get('dense_units')}
# Optimal optimizer: {best_hps.get('optimizer')}
# """)

In [39]:
# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the final model
history = model.fit(X_train, Y_train, 
                    epochs=20, 
                    validation_data=(X_test, Y_test), 
                    batch_size=64, 
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Epoch 1/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.5151 - loss: 1.1846 - val_accuracy: 0.5664 - val_loss: 0.9636
Epoch 2/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.5801 - loss: 0.9237 - val_accuracy: 0.6993 - val_loss: 0.7619
Epoch 3/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.6612 - loss: 0.7386 - val_accuracy: 0.7413 - val_loss: 0.6338
Epoch 4/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.7384 - loss: 0.6214 - val_accuracy: 0.7273 - val_loss: 0.6174
Epoch 5/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.7433 - loss: 0.5694 - val_accuracy: 0.7413 - val_loss: 0.5646
Epoch 6/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.7774 - loss: 0.4951 - val_accuracy: 0.7762 - val_loss: 0.5396
Epoch 7/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.8020 - loss: 0.4532 - val_accuracy: 0.7692 - val_loss: 0.5249
Epoch 8/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.8327 - loss: 0.4171 - val_accuracy: 0.7832 - v

In [40]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f"Test accuracy: {test_acc:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8105 - loss: 0.5235
Test accuracy: 0.8182


In [41]:
# Build the model with the 2nd best hyperparameters
model2 = tuner.hypermodel.build(best_hps2)

# Train the final model
history = model2.fit(X_train, Y_train, 
                    epochs=20, 
                    validation_data=(X_test, Y_test), 
                    batch_size=64, 
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Epoch 1/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 6s 53ms/step - accuracy: 0.5003 - loss: 1.0412 - val_accuracy: 0.5385 - val_loss: 0.9439
Epoch 2/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.5723 - loss: 0.9133 - val_accuracy: 0.6224 - val_loss: 0.8284
Epoch 3/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.6443 - loss: 0.7945 - val_accuracy: 0.6224 - val_loss: 0.7413
Epoch 4/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.6929 - loss: 0.7019 - val_accuracy: 0.6573 - val_loss: 0.6695
Epoch 5/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.7544 - loss: 0.6018 - val_accuracy: 0.7133 - val_loss: 0.6145
Epoch 6/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.7593 - loss: 0.5455 - val_accuracy: 0.7343 - val_loss: 0.5865
Epoch 7/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.8107 - loss: 0.4805 - val_accuracy: 0.7273 - val_loss: 0.5839
Epoch 8/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.8360 - loss: 0.4226 - val_accuracy: 0.7343 - v

In [42]:
# Evaluate the 2nd model
test_loss, test_acc = model2.evaluate(X_test, Y_test)
print(f"Test accuracy: {test_acc:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7706 - loss: 0.5608
Test accuracy: 0.7832


In [43]:
score,acc = model.evaluate(X_test, Y_test,verbose = 2, batch_size = batch_size)
print("loss_score: %.2f" % (score))
print("accuracy: %.2f" % (acc))

3/3 - 0s - 20ms/step - accuracy: 0.8182 - loss: 0.5358
loss_score: 0.54
accuracy: 0.82


In [44]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 1s - 1s/step
1/1 - 0s - 24ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 29ms/step
1/1 - 0s - 28ms/step
1/1 - 0s - 29ms/step
1/1 - 0s - 29ms/step
1/1 - 0s - 29ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 29ms/step
1/1 - 0s - 29ms/step
1/1 - 0s - 28ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 29ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 40ms/step
1/1 - 0s - 33ms/step
1/1 - 0s - 29ms/step
1/1 - 0s - 34ms/step
1/1 - 0s - 29ms/step
1/1 - 0s - 34ms/step
1/1 - 0s - 30ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 28ms/step
1/1 - 0s - 28ms/step
1/1 - 0s - 33ms/step
1/1 - 0s - 32ms/step
1/1 - 0s - 30ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 35ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 28ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 25ms

In [45]:
twt = ['I been told you that yo folks dont care about you . Im starving and my dad deadass didnt make me a plate']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    1   79  477    7   18  559 1000
    39  275   55    7   10 1913    6    5  458  165  118   13    4 1562]]
1/1 - 0s - 391ms/step
negative


In [46]:
model.save('lstm_model.h5')